In [1]:
%AddJar https://repo1.maven.org/maven2/io/delta/delta-core_2.11/0.4.0/delta-core_2.11-0.4.0.jar

Starting download from https://repo1.maven.org/maven2/io/delta/delta-core_2.11/0.4.0/delta-core_2.11-0.4.0.jar
Finished download of delta-core_2.11-0.4.0.jar


In [2]:
import org.apache.spark.sql.delta.DeltaLog
import io.delta.tables._
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.{SaveMode, SparkSession, DataFrame}

# Create table 

In [3]:
  def createTable(data: DataFrame, tableName: String ): Unit = {
    data
      .write
      .format("delta")
      .mode(SaveMode.Overwrite)
      .save("/opt/deltalake/" + tableName)
  }

createTable: (data: org.apache.spark.sql.DataFrame, tableName: String)Unit


# Read table

In [4]:
 def readTable(tableName: String): DataFrame = {
   val df = spark
      .read
      .format("delta")
      .load("/opt/deltalake/" + tableName)
     df
  }

readTable: (tableName: String)org.apache.spark.sql.DataFrame


In [6]:
val loan = Seq(
  (2, 1000, 1000.0, "TX"), // existing loan's paid_amnt updated, loan paid in full
  (3, 2000, 0.0, "CA"))    // new loan's details
  .toDF("loan_id", "funded_amnt", "paid_amnt", "addr_state")


loan = [loan_id: int, funded_amnt: int ... 2 more fields]


[loan_id: int, funded_amnt: int ... 2 more fields]

In [7]:
createTable(loan,"loan")

In [8]:
readTable("loan").show()

+-------+-----------+---------+----------+
|loan_id|funded_amnt|paid_amnt|addr_state|
+-------+-----------+---------+----------+
|      2|       1000|   1000.0|        TX|
|      3|       2000|      0.0|        CA|
+-------+-----------+---------+----------+



In [9]:
val loanUpdates = Seq(
  (1, 1000, 1000.0, "TX"), // existing loan's paid_amnt updated, loan paid in full
  (3, 3000, 1.0, "CAA"))    // new loan's details
  .toDF("loan_id", "funded_amnt", "paid_amnt", "addr_state")


loanUpdates = [loan_id: int, funded_amnt: int ... 2 more fields]


[loan_id: int, funded_amnt: int ... 2 more fields]

In [10]:
import io.delta.tables.DeltaTable
val deltaTable = DeltaTable.forPath(spark, "/opt/deltalake/loan")

deltaTable.alias("t").merge(
  loanUpdates.alias("s"), 
  "s.loan_id = t.loan_id")
  .whenMatched.updateAll()
  .whenNotMatched.insertAll()
  .execute()


deltaTable = io.delta.tables.DeltaTable@241ef5a


io.delta.tables.DeltaTable@241ef5a

In [11]:
readTable("loan").show()

+-------+-----------+---------+----------+
|loan_id|funded_amnt|paid_amnt|addr_state|
+-------+-----------+---------+----------+
|      3|       3000|      1.0|       CAA|
|      1|       1000|   1000.0|        TX|
|      2|       1000|   1000.0|        TX|
+-------+-----------+---------+----------+

